In [1]:
import numpy as np
import copy

In [4]:
def añadir_variables_artificiales(A, b):
    vars = []
    for v in b:
        if v >= 0:
            vars.append(1)
        else:
            vars.append(-1)
    array = np.array(vars)
    # Crear una matriz diagonal
    matriz_diagonal = np.diag(array)
    # Usamos np.hstack para apilar horizontalmente
    resultado = np.hstack((A, matriz_diagonal))
    return resultado

r = añadir_variables_artificiales(A, b)

In [ ]:
def SimplexFaseII(A, b, c, B_ini, B_indices,B_inv):
    bool_acotado = True
    m,n = A.shape
    N_ini = set(i for i in range(n) if i not in B_indices) # Antes:(n+1)
    
    x_ini = np.zeros(n)  # Crear un array de ceros en lugar de una lista
    x_b = B_inv @ b  # Multiplicación de matrices en NumPy
    x_ini[B_indices] = x_b # Asignar los valores de x_b en las posiciones correspondientes

    def actualizacion_inversa(d,inv_anterior,p):
        d = d.T.tolist()
        vector_n = np.array([[-d[i]/d[p] if i != p else -1/d[p] for i in range(len(d))]])
        e = np.identity(len(d))
        e[p] = vector_n
        e = e.T
        inversa = e @ inv_anterior 
        return inversa

    iter = 1
    while True:#####################################################################################
        print("ITERACION", iter)
        B_may = A[:, B_indices]
        N_ini = sorted(N_ini)
        A_n = A[:, N_ini]
        cn = np.array([c[i] for i in N_ini])
        cb = np.array([c[i] for i in B_indices])
        z = cb @ x_b
        
        print("B_indices", B_indices)
        print("B_may", B_may)
        print("A_n", A_n)
        print("cn", cn)
        print("cb", cb)
        print("Z", z)
        print("B_inversa",B_inv)
        #costes reducidos
        r = cn - cb @ B_inv @ A_n # r corresponde a las variables de los indices N_ini
        print("coste reducido", r)
        if np.all(r >= 0):
            return B_indices, z, bool_acotado, B_may, B_inv
        i = 0
        minimo = float('inf')
        for j in range(len(r)):
            if r[j] < minimo:
                minimo = r[j]
                i = j
        q = N_ini[i] # variable q tiene que entrar en la base 
        print("Variable q de entrada", i, "x", q,"sale de la base N")
        #direccion de descenso 
        d_b = -B_inv @ A[:, q]
        print("direccion de descenso", d_b)
        if np.all(d_b >= 0):
            print("PL NO ACOTADO")
            bool_acotado = False
            return None, None, bool_acotado, None, None
        d = np.zeros(n)  
        d[B_indices] = d_b
        d[q] = 1
        print("d", d)
    
        p = 0
        theta = float('inf')
        for w in range(len(d_b)):
            if d_b[w] < 0:
                val = -x_b[w]/d_b[w]
                if val < theta:
                    theta, p = val, w
                elif val == theta:  # Si hay empate, elegimos el menor índice en B_indices (regla de Blant)
                    if B_indices[w] < B_indices[p]:  
                        p = w
        print("Theta", theta)
        print("Variable de salida p", p, "X", B_indices[p], "sale de la base B")
        print("x_ini antes", x_ini)
        #Actualizaciones
        x_ini = x_ini + theta*d
        z = z + theta * r[i] 
        var_saliente, B_indices[p] = B_indices[p], q
        N_ini.remove(q) 
        N_ini.append(var_saliente)
        x_b = x_ini[B_indices]
        B_inv = actualizacion_inversa(d_b,B_inv,p)
        print("x_ini despues", x_ini)
        print("x_b", x_b)
        print("new_z", z)
        print("Variable de salida", var_saliente)
        print("B resultante", B_indices)
        print("N_ini", N_ini)
        iter += 1


In [ ]:
def SimplexFaseI(A, b, c):
    m, n = A.shape #las filas son m (el numero de restr.) y n son las columnas (num de variables)
    A_inicial = añadir_variables_artificiales(A, b)
    b_inicial = b 
    c_inicial = [0]*n + [1]*m 
    B_inicial = A_inicial[:,n:]
    B_inversa = B_inicial
    B_indices = list(i for i in range(n, n+m)) 
    print("Fase INICIAL", A_inicial,b_inicial,c_inicial,B_inicial, B_indices)
    B_indices_II, z_inicial, f,B_ini_II, B_inversa  = SimplexFaseII(A_inicial,b_inicial,c_inicial,B_inicial, B_indices, B_inversa)
    if (z_inicial != 0):
        print("NO FACTIBLE z no es 0 despues de la fase inicial")
        return None,None,None,False #sera no factible
    print("respuesta solo fase inicial", B_indices_II, z_inicial, f,B_ini_II)
    print()
    print()
    print()

    B_indices_opt, z , bool_acotado, B_optima, _ = SimplexFaseII(A,b,c,B_ini_II,B_indices_II,B_inversa)

    return B_optima, z , bool_acotado, True



## Problemas de Taisiia

In [12]:
# Vector c
c1 = np.array([-28, -4, 33, 51, -35, -51, -8, -49, 1, 3, -71, -20, -64, -68, 0, 0, 0, 0, 0, 0])

# Matriz A
A1 = np.array([
    [78, 92, 5, 49, -89, 45, -31, 64, 89, 61, 12, 6, -35, -88, 0, 0, 0, 0, 0, 0],
    [65, -100, 97, -31, 65, 64, 41, 39, -36, 64, 91, -84, -100, -40, 0, 0, 0, 0, 0, 0],
    [51, -15, -36, 56, -9, -7, 19, 67, -23, 88, -2, 93, -74, -38, 0, 0, 0, 0, 0, 0],
    [-25, -16, 11, 88, 70, -69, -19, -8, 63, -6, 19, -98, 75, -30, 0, 0, 0, 0, 0, 0],
    [74, 67, 53, 55, 90, 96, 76, 68, 68, 70, 86, 84, 94, 97, 1, 0, 0, 0, 0, 0],
    [-54, 6, -72, 65, 71, 12, 0, -62, -60, 56, 82, 81, -83, -6, 0, 1, 0, 0, 0, 0],
    [59, 48, -16, 61, -65, -58, -3, -46, 80, 74, 74, 74, -8, 1, 0, 0, 1, 0, 0, 0],
    [-42, 97, -53, -74, 55, -5, 10, -99, 58, -44, 47, 66, 0, -4, 0, 0, 0, -1, 0, 0],
    [-25, 20, 57, 45, 56, 90, -6, -73, -76, 95, 67, -27, -30, -65, 0, 0, 0, 0, -1, 0],
    [98, 80, -58, 81, -4, -13, -24, -24, 7, -71, 38, 44, 71, -69, 0, 0, 0, 0, 0, -1]
])

# Vector b
b1 = np.array([258, 135, 170, 55, 1079, 37, 276, 11, 127, 155])

opt, zeta, acotado, bool_res = SimplexFaseI(A1,b1,c1)
print(opt,zeta,acotado,bool_res)

Fase INICIAL [[  78   92    5   49  -89   45  -31   64   89   61   12    6  -35  -88
     0    0    0    0    0    0    1    0    0    0    0    0    0    0
     0    0]
 [  65 -100   97  -31   65   64   41   39  -36   64   91  -84 -100  -40
     0    0    0    0    0    0    0    1    0    0    0    0    0    0
     0    0]
 [  51  -15  -36   56   -9   -7   19   67  -23   88   -2   93  -74  -38
     0    0    0    0    0    0    0    0    1    0    0    0    0    0
     0    0]
 [ -25  -16   11   88   70  -69  -19   -8   63   -6   19  -98   75  -30
     0    0    0    0    0    0    0    0    0    1    0    0    0    0
     0    0]
 [  74   67   53   55   90   96   76   68   68   70   86   84   94   97
     1    0    0    0    0    0    0    0    0    0    1    0    0    0
     0    0]
 [ -54    6  -72   65   71   12    0  -62  -60   56   82   81  -83   -6
     0    1    0    0    0    0    0    0    0    0    0    1    0    0
     0    0]
 [  59   48  -16   61  -65  -58   -3  -46   8

In [11]:
c2 = np.array([83, -62, -65, -49, 27, -78, -73, -83, -29, -17, -43, 42, 76, 2, 0, 0, 0, 0, 0, 0])

# Matriz A
A2 = np.array([
    [3, -8, -66, 27, -45, 0, -47, 99, -5, 66, 71, -64, 86, 51, 0, 0, 0, 0, 0, 0],
    [16, 93, -5, 88, -56, 81, 28, 10, 43, -12, 98, 14, -54, 1, 0, 0, 0, 0, 0, 0],
    [-82, 36, 45, 61, -50, -90, -38, 32, 8, -21, 50, 84, -6, 80, 0, 0, 0, 0, 0, 0],
    [-28, 18, 72, -86, -13, 91, 37, 15, 91, 98, 69, -97, -2, 20, 0, 0, 0, 0, 0, 0],
    [19, -50, 39, 61, -23, -16, -35, 68, 18, 51, 56, -13, -69, 81, 1, 0, 0, 0, 0, 0],
    [-84, 42, -73, -57, 91, -50, -32, 47, 86, -59, 35, 21, 48, 53, 0, -1, 0, 0, 0, 0],
    [52, 39, 22, -49, -36, -39, 67, 32, 48, 77, 66, -51, -2, -94, 0, 0, 1, 0, 0, 0],
    [66, 91, 0, 14, 4, 56, 35, 15, 15, 10, 20, 63, 73, 57, 0, 0, 0, 1, 0, 0],
    [27, 21, 98, 38, 23, 33, 91, 38, 59, 69, 21, 53, 87, 41, 0, 0, 0, 0, 1, 0],
    [5, 69, 83, 55, 12, 68, 25, 28, 28, 62, 14, 27, 87, 53, 0, 0, 0, 0, 0, 1]
])

# Vector b
b2 = np.array([168, 345, 109, 285, 188, 67, 133, 30, 80, 62])
opt, zeta, acotado, bool_res = SimplexFaseI(A2,b2,c2)
print(opt,zeta,acotado,bool_res)

Fase INICIAL [[  3  -8 -66  27 -45   0 -47  99  -5  66  71 -64  86  51   0   0   0   0
    0   0   1   0   0   0   0   0   0   0   0   0]
 [ 16  93  -5  88 -56  81  28  10  43 -12  98  14 -54   1   0   0   0   0
    0   0   0   1   0   0   0   0   0   0   0   0]
 [-82  36  45  61 -50 -90 -38  32   8 -21  50  84  -6  80   0   0   0   0
    0   0   0   0   1   0   0   0   0   0   0   0]
 [-28  18  72 -86 -13  91  37  15  91  98  69 -97  -2  20   0   0   0   0
    0   0   0   0   0   1   0   0   0   0   0   0]
 [ 19 -50  39  61 -23 -16 -35  68  18  51  56 -13 -69  81   1   0   0   0
    0   0   0   0   0   0   1   0   0   0   0   0]
 [-84  42 -73 -57  91 -50 -32  47  86 -59  35  21  48  53   0  -1   0   0
    0   0   0   0   0   0   0   1   0   0   0   0]
 [ 52  39  22 -49 -36 -39  67  32  48  77  66 -51  -2 -94   0   0   1   0
    0   0   0   0   0   0   0   0   1   0   0   0]
 [ 66  91   0  14   4  56  35  15  15  10  20  63  73  57   0   0   0   1
    0   0   0   0   0   0   0   0   0 

In [10]:
# Vector c
c3 = np.array([-86, -76, -64, -49, -36, -49, -1, -79, -82, -99, 0, -95, -18, -95, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Matriz A
A3 = np.array([
    [28, 49, 53, 65, 75, 50, 64, 50, 98, 10, 79, 64, 46, 80, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [27, 36, 66, 90, 88, 88, 15, 35, 52, 26, 81, 5, 54, 96, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0],
    [26, 85, 0, 20, 53, 45, 20, 99, 74, 97, 24, 66, 52, 34, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0],
    [2, 38, 1, 93, 58, 17, 23, 93, 30, 65, 6, 73, 96, 65, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0],
    [83, 96, 5, 3, 98, 15, 34, 70, 73, 5, 1, 86, 8, 49, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0],
    [86, 31, 62, 64, 5, 14, 82, 5, 48, 60, 62, 60, 64, 88, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0],
    [31, 80, 32, 69, 27, 57, 68, 96, 33, 64, 48, 45, 65, 99, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0],
    [91, 36, 20, 98, 97, 3, 8, 67, 50, 83, 98, 46, 93, 42, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0],
    [95, 74, 60, 53, 5, 95, 12, 65, 55, 82, 50, 80, 82, 75, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0],
    [85, 75, 23, 71, 4, 84, 27, 98, 48, 72, 31, 65, 53, 98, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]
])

# Vector b
b3 = np.array([810, 758, 694, 659, 625, 730, 813, 831, 882, 833])

opt, zeta, acotado, bool_res = SimplexFaseI(A3,b3,c3)
print(opt,zeta,acotado,bool_res)


Fase INICIAL [[28 49 53 65 75 50 64 50 98 10 79 64 46 80 -1  0  0  0  0  0  0  0  0  0
   1  0  0  0  0  0  0  0  0  0]
 [27 36 66 90 88 88 15 35 52 26 81  5 54 96  0 -1  0  0  0  0  0  0  0  0
   0  1  0  0  0  0  0  0  0  0]
 [26 85  0 20 53 45 20 99 74 97 24 66 52 34  0  0 -1  0  0  0  0  0  0  0
   0  0  1  0  0  0  0  0  0  0]
 [ 2 38  1 93 58 17 23 93 30 65  6 73 96 65  0  0  0 -1  0  0  0  0  0  0
   0  0  0  1  0  0  0  0  0  0]
 [83 96  5  3 98 15 34 70 73  5  1 86  8 49  0  0  0  0 -1  0  0  0  0  0
   0  0  0  0  1  0  0  0  0  0]
 [86 31 62 64  5 14 82  5 48 60 62 60 64 88  0  0  0  0  0 -1  0  0  0  0
   0  0  0  0  0  1  0  0  0  0]
 [31 80 32 69 27 57 68 96 33 64 48 45 65 99  0  0  0  0  0  0 -1  0  0  0
   0  0  0  0  0  0  1  0  0  0]
 [91 36 20 98 97  3  8 67 50 83 98 46 93 42  0  0  0  0  0  0  0 -1  0  0
   0  0  0  0  0  0  0  1  0  0]
 [95 74 60 53  5 95 12 65 55 82 50 80 82 75  0  0  0  0  0  0  0  0 -1  0
   0  0  0  0  0  0  0  0  1  0]
 [85 75 23 71  4 84 27 9

## Problemas de Pablo:

In [15]:
# Vector c

c_1 = np.array([5,18,17,79,-70,27,80,92,64,-85,-28,58,-25,-60,0,0,0,0,0,0])

# Matriz A
a_1 = np.array([
    [81,-96,75,25,-82,85,12,15,75,32,-76,23,-10,22,0,0,0,0,0,0],
    [21,21,-50,90,62,-21,-27,-66,-26,74,47,-22,-11,86,0,0,0,0,0 ,0],
    [100,34, -97,3,99,77,80,-80,44,-87,26,-98,-53,25,0,0,0,0,0,0],
    [90, -53,24, -81, -48,54,-1,-7,18,-13,-21,64,-19,28,0,0,0,0,0,0],
    [84,100,87,78,60,54,91,66,78,65,91,82,58,79,1,0,0,0,0,0],
    [-57,65,-42,-97,-27,80,59,14,19,-23,-8,34,65,-25,0,1,0,0,0,0],
    [97,-8,40,86,-13,46,30,75,-33,-69,97,66,82,56,0, 0, 1, 0, 0,0],
    [-28,70,34,80,-85,38,-52,94,-23,78,95,-52,92,74,0, 0,0,-1,0,0],
    [-23,42,-75,61,-65,-33,56,59,-43,29,87,76,-15,35,0,0,0,0,1,0],
    [92,70,74,30,-63,76,-71,48,-50,-72,-93,-55,50,-29,0,0,0,0,0,-1]
])

# Vector b
b_1 = np.array([181,178,73,35,1074,58,553,414,192,6])

opt, zeta, acotado, bool_res = SimplexFaseI(a_1,b_1,c_1)
print(opt,zeta,acotado,bool_res)

Fase INICIAL [[ 81 -96  75  25 -82  85  12  15  75  32 -76  23 -10  22   0   0   0   0
    0   0   1   0   0   0   0   0   0   0   0   0]
 [ 21  21 -50  90  62 -21 -27 -66 -26  74  47 -22 -11  86   0   0   0   0
    0   0   0   1   0   0   0   0   0   0   0   0]
 [100  34 -97   3  99  77  80 -80  44 -87  26 -98 -53  25   0   0   0   0
    0   0   0   0   1   0   0   0   0   0   0   0]
 [ 90 -53  24 -81 -48  54  -1  -7  18 -13 -21  64 -19  28   0   0   0   0
    0   0   0   0   0   1   0   0   0   0   0   0]
 [ 84 100  87  78  60  54  91  66  78  65  91  82  58  79   1   0   0   0
    0   0   0   0   0   0   1   0   0   0   0   0]
 [-57  65 -42 -97 -27  80  59  14  19 -23  -8  34  65 -25   0   1   0   0
    0   0   0   0   0   0   0   1   0   0   0   0]
 [ 97  -8  40  86 -13  46  30  75 -33 -69  97  66  82  56   0   0   1   0
    0   0   0   0   0   0   0   0   1   0   0   0]
 [-28  70  34  80 -85  38 -52  94 -23  78  95 -52  92  74   0   0   0  -1
    0   0   0   0   0   0   0   0   0 

In [ ]:
# Vector c
c_2 = np.array([-85,-4,84,17,-8,0,53,58,34,96,-36,-43,-86,10,0,0,0,0,0,0])

# Matriz A
a_2 = np.array([
    [39,22,99,-62,-80,97,1,53,-22,74,12,-69,-100,99,0,0,0,0,0,0],
    [-66,71,43,96,-42,-53,47,-7,35,61,70,-66,34,49,0,0,0,0,0,0],
    [-71,45,-47,-38,59,69,-92,-97,8,27,61,-15,6,92,0,0,0,0,0,0],
    [-16,-40,-74,59,90,46,79,70,96,90,-27,48,-33,72,0,0,0,0,0,0],
    [94,97,89,88,84,93,91,81,59,80,86,98,84,71,1,0,0,0,0,0,0],
    [-82,54,-72,39,-97,100,38,79,61,2,89,73,47,-16,0,1,0,0,0,0],
    [-52,11,86,95,21,64,65,52,-82,31,66,55,-52,-7,0,0,-1,0,0,0],
    [-6,61,0,63,-91,-84,11,70,79,-28,-13,6,67,-18,0,0,0,1,0,0],
    [-67,23,-11,-4,-40,87,27,-28,-52,8,42,96,56,-71,0,0,0,0,1,0],
    [30,63,76,96,40,-2,30,-45,96,78,-5,-79,32,43,0,0,0,0,0,1]
])

# Vector b
b_2 = np.array([163,272,7,460,1196,316,352,118,67,452])

opt, zeta, acotado, bool_res = SimplexFaseI(a_2,b_2,c_2)
print(opt,zeta,acotado,bool_res)

Fase INICIAL [[  39   22   99  -62  -80   97    1   53  -22   74   12  -69 -100   99
     0    0    0    0    0    0    1    0    0    0    0    0    0    0
     0    0]
 [ -66   71   43   96  -42  -53   47   -7   35   61   70  -66   34   49
     0    0    0    0    0    0    0    1    0    0    0    0    0    0
     0    0]
 [ -71   45  -47  -38   59   69  -92  -97    8   27   61  -15    6   92
     0    0    0    0    0    0    0    0    1    0    0    0    0    0
     0    0]
 [ -16  -40  -74   59   90   46   79   70   96   90  -27   48  -33   72
     0    0    0    0    0    0    0    0    0    1    0    0    0    0
     0    0]
 [  94   97   89   88   84   93   91   81   59   80   86   98   84   71
     1    0    0    0    0    0    0    0    0    0    1    0    0    0
     0    0]
 [ -82   54  -72   39  -97  100   38   79   61    2   89   73   47  -16
     0    1    0    0    0    0    0    0    0    0    0    1    0    0
     0    0]
 [ -52   11   86   95   21   64   65   52  -8

In [18]:
# Vector c
c_3 = np.array([26,-62,-50,-83,54,-27,-12,-12,4,76,-8,19,90,-85,0,0,0,0,0,0])

# Matriz A
a_3 = np.array([
    [42,4,57,-27,74,10,21,-58,25,-30,-83,-35,81,48,0,0,0,0,0,0],
    [79,16,-43,-77,-78,75,-69,78,69,-36,-52,86,98,24,0,0,0,0,0,0],
    [33,-68,-14,88,25,-79,-23,-85,68,53,51,63,-4,32,0,0,0,0,0,0],
    [96,-52,29,28,25,-69,4,-61,80,-41,-70,-43,99,23,0,0,0,0,0,0],
    [-24,-80,-4,25,84,-37,45,91,-14,-6,-22,58,23,27,1,0,0,0,0,0],
    [56,54,56,-13,-37,41,68,-64,-80,38,-59,10,41,43,0,1,0,0,0,0],
    [45,95,-72,-32,-65,55,51,-81,-63,95,60,-9,21,-22,0,0,1,0,0,0],
    [87,72,85,59,93,69,83,26,58,24,6,37,56,70,0,0,0,1,0,0],
    [91,72,62,15,73,60,35,52,42,69,59,62,32,90,0,0,0,0,1,0],
    [10,100,30,77,90,49,14,21,77,44,58,88,43,97,0,0,0,0,0,1]
])

# Vector b
b_3 = np.array([129,170,140,48,167,155,79,72,11,78])

opt, zeta, acotado, bool_res = SimplexFaseI(a_3,b_3,c_3)
print(opt,zeta,acotado,bool_res)

Fase INICIAL [[ 42   4  57 -27  74  10  21 -58  25 -30 -83 -35  81  48   0   0   0   0
    0   0   1   0   0   0   0   0   0   0   0   0]
 [ 79  16 -43 -77 -78  75 -69  78  69 -36 -52  86  98  24   0   0   0   0
    0   0   0   1   0   0   0   0   0   0   0   0]
 [ 33 -68 -14  88  25 -79 -23 -85  68  53  51  63  -4  32   0   0   0   0
    0   0   0   0   1   0   0   0   0   0   0   0]
 [ 96 -52  29  28  25 -69   4 -61  80 -41 -70 -43  99  23   0   0   0   0
    0   0   0   0   0   1   0   0   0   0   0   0]
 [-24 -80  -4  25  84 -37  45  91 -14  -6 -22  58  23  27   1   0   0   0
    0   0   0   0   0   0   1   0   0   0   0   0]
 [ 56  54  56 -13 -37  41  68 -64 -80  38 -59  10  41  43   0   1   0   0
    0   0   0   0   0   0   0   1   0   0   0   0]
 [ 45  95 -72 -32 -65  55  51 -81 -63  95  60  -9  21 -22   0   0   1   0
    0   0   0   0   0   0   0   0   1   0   0   0]
 [ 87  72  85  59  93  69  83  26  58  24   6  37  56  70   0   0   0   1
    0   0   0   0   0   0   0   0   0 

In [22]:
# Vector c
c_4 = np.array([-70,-57,-73,-64,-7,-95,-90,-14,-36,-5,-16,-98,-4,-36,0,0,0,0,0,0,0,0,0,0])

# Matriz A
a_4 = np.array([
    [94,74,3,43,18,60,85,35,6,72,74,27,32,36,-1,0,0,0,0,0,0,0,0,0],
    [18,39,45,38,42,51,81,25,28,36,48,56,9,21,0,-1,0,0,0,0,0,0,0,0],
    [83,44,34,10,71,49,54,36,62,94,74,37,9,77,0,0,-1,0,0,0,0,0,0,0],
    [11,39,39,91,33,81,90,87,64,34,76,60,12,4,0,0,0,-1,0,0,0,0,0,0],
    [22,71,38,29,52,1,77,9,39,71,17,59,60,66,0,0,0,0,-1,0,0,0,0,0],
    [53,33,68,67,75,86,50,8,54,7,46,30,50,42,0,0,0,0,0,-1,0,0,0,0],
    [79,85,73,64,91,33,42,48,5,36,0,46,65,42,0,0,0,0,0,0,-1,0,0,0],
    [58,1,37,24,25,20,79,83,47,14,4,1,59,29,0,0,0,0,0,0,0,-1,0,0],
    [6,48,87,53,72,62,4,34,13,61,73,91,96,26,0,0,0,0,0,0,0,0,-1,0],
    [83,14,68,67,98,14,78,11,68,8,9,54,15,5,0,0,0,0,0,0,0,0,0,-1]
])

# Vector b
b_4 = np.array([658,536,733,720,610,668,708,480,725,591])

opt, zeta, acotado, bool_res = SimplexFaseI(a_4,b_4,c_4)
print(opt,zeta,acotado,bool_res)

Fase INICIAL [[94 74  3 43 18 60 85 35  6 72 74 27 32 36 -1  0  0  0  0  0  0  0  0  0
   1  0  0  0  0  0  0  0  0  0]
 [18 39 45 38 42 51 81 25 28 36 48 56  9 21  0 -1  0  0  0  0  0  0  0  0
   0  1  0  0  0  0  0  0  0  0]
 [83 44 34 10 71 49 54 36 62 94 74 37  9 77  0  0 -1  0  0  0  0  0  0  0
   0  0  1  0  0  0  0  0  0  0]
 [11 39 39 91 33 81 90 87 64 34 76 60 12  4  0  0  0 -1  0  0  0  0  0  0
   0  0  0  1  0  0  0  0  0  0]
 [22 71 38 29 52  1 77  9 39 71 17 59 60 66  0  0  0  0 -1  0  0  0  0  0
   0  0  0  0  1  0  0  0  0  0]
 [53 33 68 67 75 86 50  8 54  7 46 30 50 42  0  0  0  0  0 -1  0  0  0  0
   0  0  0  0  0  1  0  0  0  0]
 [79 85 73 64 91 33 42 48  5 36  0 46 65 42  0  0  0  0  0  0 -1  0  0  0
   0  0  0  0  0  0  1  0  0  0]
 [58  1 37 24 25 20 79 83 47 14  4  1 59 29  0  0  0  0  0  0  0 -1  0  0
   0  0  0  0  0  0  0  1  0  0]
 [ 6 48 87 53 72 62  4 34 13 61 73 91 96 26  0  0  0  0  0  0  0  0 -1  0
   0  0  0  0  0  0  0  0  1  0]
 [83 14 68 67 98 14 78 1